# Impianti sportivi

In [2]:
import json
from datetime import datetime

In [3]:
PATH_IMPIANTI_SPORTIVI_RAW = "../Data/Raw/Punti_di_interesse/impianti_sportivi_raw.geojson"
PATH_IMPIANTI_SPORTIVI_CLEAN = "../Data/Clean/Punti_di_interesse/impianti_sportivi_clean.geojson"

## ETL

1. Rimozione righe in cui non è segnalata una data inizio costruzione
2. Rimozione delle piste ciclabili, dato che abbiamo un dataset apposito
3. Convertiamo la data in un intero, in modo da poter filtrare agilmente su kepler

In [ ]:
with open(PATH_IMPIANTI_SPORTIVI_RAW, "r", encoding="utf-8") as f:
    data = json.load(f)

new_features = []
for feature in data["features"]:
    props = feature["properties"]

    # Verifica che esista una data di inizio
    if props["data_ini"]:
        # Controlla che la descrizione non contenga "pista ciclabile" (ignorando maiuscole/minuscole)
        descrizione = props["descrizione_codice"].lower()
        if "pista ciclabile" not in descrizione:
            # Convertiamo data da stringa a datetime e preleviamo solo l'anno come intero
            anno = datetime.strptime(props["data_ini"], "%Y-%m-%d").year
            # Crea una nuova feature con le proprietà pulite
            new_features.append({
                "type": "Feature",
                "properties": {
                    "municipio": props["municipio"],
                    "località": props["località"],
                    "id_impianto": props["obj_id"],
                    "anno_inizio": anno,
                    "area_mq": props["area_mq"],
                    "perim_m": props["perim_m"],
                    "descrizione_codice": props["descrizione_codice"]
                },
                "geometry": feature["geometry"]
            })

# Sovrascrive le feature nel dizionario con quelle filtrate
data["features"] = new_features

In [5]:
with open(PATH_IMPIANTI_SPORTIVI_CLEAN, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)